# Resume AI Agent Jupyter Notebook

## TEXT HERE

In [12]:
import os
from dotenv import load_dotenv

try:
    load_dotenv()
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"API Authentication Error, please make sure you have setup your .env with the correct GOOGLE_API_KEY: {e}")

✅ Gemini API key setup complete.


In [2]:
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [8]:
async def run_session(
    runner_instance: Runner, user_queries: list[str] | str, session_id: str = "default"
):
    """Helper function to run queries in a session and display responses."""
    print(f"\n### Session: {session_id}")

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )

    # Convert single query to list
    if isinstance(user_queries, str):
        user_queries = [user_queries]

    # Process each query
    for query in user_queries:
        print(f"\nUser > {query}")
        query_content = types.Content(role="user", parts=[types.Part(text=query)])

        # Stream agent response
        async for event in runner_instance.run_async(
            user_id=USER_ID, session_id=session.id, new_message=query_content
        ):
            if event.is_final_response() and event.content and event.content.parts:
                text = event.content.parts[0].text
                if text and text != "None":
                    print(f"Model: > {text}")

async def auto_save_to_memory(callback_context):
    """Automatically save session to memory after each agent turn."""
    await callback_context._invocation_context.memory_service.add_session_to_memory(
        callback_context._invocation_context.session
    )

    
print("✅ Helper functions defined.")

✅ Helper functions defined.


In [4]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [5]:
memory_service = (
    InMemoryMemoryService()
)  # ADK's built-in Memory Service for development and testing

In [10]:
# Define constants used throughout the notebook
APP_NAME = "ResumeReviewApp"
USER_ID = "user"

# Create agent
resume_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="ResumeReviewAgent",
    instruction="Review the given resume and provide feedback on grammar, structure," + 
    " action-verb oriented bullet points, missing metrics (if any), and general improvements",
    tools = [preload_memory], 
    after_agent_callback = auto_save_to_memory,
)

print("✅ Agent created")

✅ Agent created


In [ ]:
runner = Runner(
    agent = resume_agent,
    app_name = APP_NAME,
    session_service = session_service,
    memory_service = memory_service,
)

